# 훈련용 데이터셋 context 확인
- 훈련용 데이터셋에 있는 context들이 주어진 `wikipedia_documents.json`에 있는지 속하는지 확인한다.

먼저, 훈련용 데이터셋에 있는 context를 불러온다.

In [2]:
from datasets import load_from_disk
import pandas as pd

In [3]:
data_path = '../data/train_dataset/'
data = load_from_disk(data_path)

In [4]:
train_data = data['train']

In [42]:
train_context = pd.Series(train_data['context'], name='context')

wikipedia_documents에 있는 context를 불러온다.

In [6]:
import json

In [7]:
wiki_path = '../data/wikipedia_documents.json'
with open(wiki_path) as f:
    wiki = json.load(f)
wiki = pd.DataFrame(wiki).T

In [19]:
merged = pd.merge(train_context, wiki, left_on='context', right_on='text', how='inner')
merged.drop(columns=['text'], inplace=True)

In [20]:
print("merged:", len(merged))
print("wiki:", len(wiki))
print("train_context:", len(train_context))

merged: 3984
wiki: 60613
train_context: 3952


`merged`의 결과가 `min(wiki, train_context)`보다도 더 크다?  
natural join의 결과가 두 테이블의 최소 길이보다 클 수는 없는데... 무슨 문제가 있는 것 같다.

In [46]:
b = merged['context'][:len(train_context)] == train_context
b.loc[b == False][:10]

323    False
324    False
325    False
326    False
327    False
328    False
329    False
330    False
331    False
332    False
Name: context, dtype: bool

In [47]:
b[320:325]
# 쭉 같다가 322번째부터 다르다.

320     True
321     True
322     True
323    False
324    False
Name: context, dtype: bool

In [48]:
merged[320:325]

,context,corpus_source,url,domain,title,author,html,document_id
320,1878년 10월 21일 조선 충청북도 증평에서 태어난 그는 형제로는 독립운동가 연...,위키피디아,None,None,연병환,None,None,53017
321,"이오에 대한 탐사\n 이오의 발견을 최초로 공표한 사람은 갈릴레오 갈릴레이로, 16...",위키피디아,None,None,이오 (위성),None,None,8264
322,1544년 중종이 병사하였다. 그러나 보위를 재빨리 인종이 접수함으로써 경원대군 추...,위키피디아,TODO,None,문정왕후 (조선),None,None,2444
323,1544년 중종이 병사하였다. 그러나 보위를 재빨리 인종이 접수함으로써 경원대군 추...,위키피디아,None,None,문정왕후 (조선),None,None,6998
324,아게실라오스는 기원전 245년에 왕위에 오른 아기스 4세의 어머니 아게시스토라타의 ...,위키피디아,None,None,아게실라오스,None,None,45562


322번 context와 323번 context가 서로 같은데 document-id가 다르다.  
아마 `wikipedia_documents.json`에는 같은 문서가 여러 개 들어가 있는 경우가 있는 것 같다.

In [51]:
wiki = wiki.drop_duplicates(subset='text', keep='first') # 중복 제거

In [52]:
merged = pd.merge(train_context, wiki, left_on='context', right_on='text', how='inner')
merged.drop(columns=['text'], inplace=True)

In [54]:
print("merged:", len(merged))
print("wiki:", len(wiki))
print("train_context:", len(train_context))

merged: 3952
wiki: 56737
train_context: 3952


3876개의 중복된 text가 wikipedia_documents에 있었다.  
이를 제거하고 join하니 값이 train_context와 같아진 걸 확인했다.  

In [ ]:
b = merged['context'][:len(train_context)] == train_context
b.loc[b == False][:10]

323    False
324    False
325    False
326    False
327    False
328    False
329    False
330    False
331    False
332    False
Name: context, dtype: bool

# 결론
1. `wikipedia_documents.json`에는 중복된 문서도 존재한다. (3678개)
2. `train_dataset`에서 사용되는 `context`는 모두 `wikipedia_documents.json`에서 확인할 수 있다.
3. 단, `wikipedia_documents.json`에 존재하는 context 중, $10\%$도 활용하지 못하고 있다.